In [167]:
# Takes the output of phase2assignment and builds segments based on the pairings made
# pairing: bunch of tuples (carry1,prep1); (carry2, prep1)
# segments: a list of sites starting at a prep site and ending at some carry in site
# no more than 2 carry ins per site

In [168]:
%run ./phase2assignment.ipynb
# Comment out in phase2assignment whatever you don't want re-printing here
# runs phase2 so that all of those outputs/functions are easily used here 

Total distance (km) = 132.68026911860102


In [169]:
## Copied from Aarij's code from phase I 

# Creating a distance function from the distance matrix
# Given two school's 3 digit codes, it returns the distance in time between them
# Ex: distance(981, 604) = 35.736
timematrix = pd.read_csv("C://Users/anyak/Documents/DPS/DPSData/time_matrix.csv")

origin      = list(timematrix['Origin_School'])
destination = list(timematrix['Dest_School'])
times       = list(timematrix['Total_TravelTime'])

pairs = zip(origin, destination, times)
distance_dictionary = {}
for src, dst, time in pairs:
    distance_dictionary[(src, dst)] = time

In [170]:
# Helpers
def distance(src_, dst_): 
# Edited from Aarij's to accommodate for 4 character codings
    if len(str(src_)) == 4:
        temp_src = str(src_)
        temp_src = temp_src[0] + temp_src[1] + temp_src[2] # definitely a better way to do this... 
        src_ = int(temp_src)
    if len(str(dst_)) == 4:
        temp_dst = str(dst_)
        temp_dst = temp_dst[0] + temp_dst[1] + temp_dst[2]
        dst_ = int(temp_dst)
    return distance_dictionary[(src_, dst_)]

def demand_sum(carry_list):
# Total demand of the sites in any given sized list of carry_ins
    demand = 0
    for c in carry_ins:
        demand = demand + carry_in_dict[c]
    return demand

def delete_ending(any_list):
# deletes the last character of each 4 character entry in a list 
    for i in range(len(any_list)): 
        site_str = str(any_list[i])
        
        if len(site_str) == 4:
            site_str = site_str[0] + site_str[1] + site_str[2]
            
        site_num = int(site_str)
        any_list[i] = site_num
    
    return any_list

def same_meal_time(carry_site1,carry_site2):
# 1 (true) if the meal times are the same 
    str1 = str(carry_site1)
    str2 = str(carry_site2)
    len1 = len(str1)
    len2 = len(str2)
    
    same_meal = 0 #assume they are different meals unless otherwise proven (cuts down on else statements)
    
    if len1 == 4 and len2 == 4:
        if str1[3] == str2[3]: # if both are 4 digits with the same ending, same meal time
            same_meal = 1
    
    if len1 == 4 and len2 == 3:
        if str1[3] == 1:
            same_meal = 1
            
    if len1 == 3 and len2 == 4:
        if str2[3] == 1:
            same_meal = 1

    if len1 == 3 and len2 == 3:
        same_meal = 1
        
    return same_meal

In [171]:
# Variables
vehicle_capacity = 800
max_segment_length = 2

In [172]:
prep_list # check helper functions in phase2assignment for details

{451: [110, 516, 3831, 4790, 4791, 6040, 6041],
 450: [158, 2121],
 190: [161],
 419: [168, 328],
 437: [178, 252, 473, 509],
 415: [179, 192],
 424: [203, 4880, 4881, 4990, 4991, 5320, 5321],
 971: [223, 533],
 214: [264],
 455: [426, 1170],
 981: [436, 477],
 457: [438],
 150: [497, 891, 5150, 5151],
 412: [605, 5220, 5221],
 292: [750, 3160],
 301: [801],
 405: [999],
 982: [1171],
 408: [1810, 1811],
 218: [2130, 2131, 4780, 4781],
 258: [3161, 3830],
 423: [4900, 4901],
 278: [6020, 6021]}

In [173]:
# Another 'helper'
def segment_optimization(prep, list_of_carry_ins):
# basically running optimization on each group of prep/CI combos
# returns a dictionary in the format of prep_list

    solver = pywraplp.Solver('SolveAssignmentProblemMIP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    dummies = [0] * 4 # 4 is arbitrary
    for i in range(len(dummies)):
        dummies[i] = prep + i # (makes them distinct from each other if not from every site)
        
    carry_in_site_number = list_of_carry_ins # original list with some 4 digit entries 
    prep_site_number = dummies
    bools = {}
    
    for i in carry_in_site_number:
        for j in prep_site_number: 
            bools[(i, j)] = solver.BoolVar('bool[%i,%i]' % (i, j))
            
    # Sort of a dummy objective 
    solver.Minimize(solver.Sum([costs[(i,prep)]*bools[(i,j)] # use prep not j because j is dummy var / no data 
                                        for i in carry_in_site_number
                                        for j in prep_site_number]))
            
    for i in carry_in_site_number: 
        # carry in can only be assigned to one dummy site
        solver.Add(solver.Sum([bools[(i,j)] for j in prep_site_number]) == 1)
        
    for j in prep_site_number: 
        # a dummy prep can have max 2 carry ins assigned to it
        solver.Add(solver.Sum([bools[(i,j)] for i in carry_in_site_number]) <= 2)
        
        # a segment cannot require more than the max vehicle capacity
        solver.Add(solver.Sum([bools[(i,j)]*carry_in_dict[i] for i in carry_in_site_number]) <= vehicle_capacity)
        
    solver.Solve()
    
    seg_assignments = {}
    
    for i in carry_in_site_number:
        for j in prep_site_number:
            if bools[i, j].solution_value() > 0:
                seg_assignments[(i,j)] = 1

    return make_prep_list(seg_assignments)

In [174]:
final_segments = {}
for prep in prep_list.keys():
    carry_ins = prep_list[prep]
        
    # Case 1: single carry in site (leave as is)
    if len(carry_ins) == 1:
        final_segments[prep] = carry_ins
    
    # Case 2: 2 carry in sites 
    if len(carry_ins) == 2:
        first = carry_ins[0]
        second = carry_ins[1]
            
        # can't go over vehicle_capacity
        if carry_in_dict[first] + carry_in_dict[second] >= vehicle_capacity:
            final_segments[prep] = [[first],[second]]
                
        # if too 'far' (arbitrary?) make 2 segments
        elif distance(first,second) > distance(prep,second):
            final_segments[prep] = [[first],[second]]
                
        # if not the same meal time, make 2 segments
        elif same_meal_time(first,second) == 0:
            final_segments[prep] = [[first],[second]]
                
        else:
            final_segments[prep] = [[first,second]]
                
    # Case 3: more than 2 carry in sites 
    if len(carry_ins) > 2: 
        temp_dict = segment_optimization(prep,carry_ins)
        final_segments[prep] = []
        for val in temp_dict.values():
            final_segments[prep].append(val)

In [175]:
final_segments ## the outer list just means they are all assigned to that prep; inner list is the segment 

{451: [[110, 6041], [516, 4790], [3831, 6040], [4791]],
 450: [[158], [2121]],
 190: [161],
 419: [[168], [328]],
 437: [[178, 473], [252, 509]],
 415: [[179, 192]],
 424: [[203, 5321], [4880, 4990], [4881, 5320], [4991]],
 971: [[223], [533]],
 214: [264],
 455: [[426], [1170]],
 981: [[436, 477]],
 457: [438],
 150: [[497], [891, 5151], [5150]],
 412: [[605, 5221], [5220]],
 292: [[750], [3160]],
 301: [801],
 405: [999],
 982: [1171],
 408: [[1810], [1811]],
 218: [[2130, 4780], [2131, 4781]],
 258: [[3161], [3830]],
 423: [[4900], [4901]],
 278: [[6020], [6021]]}